In [ ]:
#Import of main libraries 
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np 
from __future__ import division, print_function
from pandas import read_excel
from pandas import DataFrame
from pandas import ExcelWriter
from pandas import ExcelFile

In [ ]:
#Import of the pyomo module (May need to pip install)
from pyomo.environ import *
#Creation of a Concrete Model
model = ConcreteModel()

In [ ]:
BidsDf = read_excel('Logistics Tendering Data for CaseStudy PIC2018.xlsx', sheet_name='Bids')

In [ ]:
BidsDf.head()

In [ ]:
BidsDf.dtypes

In [ ]:
LanesDf = read_excel('Logistics Tendering Data for CaseStudy PIC2018.xlsx', sheet_name='Lots')
LanesDf.head()

In [ ]:
#Create a data frame for delta
deltaDf = DataFrame(np.zeros((len(LanesDf.index), len(BidsDf.index))))

for bid in BidsDf.index:
    deltaDf.at[BidsDf.loc[bid,'Lot#'] - 1, bid] = 1


In [ ]:
# Create a data frame for gamma
gammaDf = DataFrame(np.zeros((len(LanesDf.index), len(BidsDf.index))))

for bid in BidsDf.index:
    gammaDf.at[BidsDf.loc[]]

In [ ]:
#Number of bids in the program
model.numBids = len(BidsDf.index)
#Number of lanes in the program
model.numItems = len(LanesDf.index)

In [ ]:
## Define sets
model.BIDS = Set(initialize = BidsDf.index.values)
model.LANES = Set(initialize = LanesDf.index.values)

In [ ]:
# Create a dictionary of the bid values
bidValues = dict()
for bid in BidsDf.index:
    bidValues[bid] = BidsDf.loc[bid, 'Cost']
bidValues

In [ ]:
#Initialize bidValue parameter with the value of each bid 
model.bidValue = Param(model.BIDS, initialize = bidValues, doc='Value of each bid in the program')

In [ ]:
#Create a dictionary of the total volume on each lane
demandValues = dict()
for lane in LanesDf.index:
    demandValues[lane] = 1

In [ ]:
#Initialize demand parameter with the total volume on each lane (rep as demand/multiplicity)
model.demand = Param(model.LANES, initialize = demandValues, doc='Total demand on each lane')

In [ ]:
#Create a dictionary of the delta values
delta = dict()
for lane in LanesDf.index:
    for bid in BidsDf.index:
        delta[(lane,bid)] = deltaDf.loc[lane,bid]
#delta

In [ ]:
#Initialize the delta parameter
model.delta = Param(model.LANES, model.BIDS, initialize=delta, doc='delta gives information regarding which lanes are in a bid package')

In [ ]:
#Create a dictionary for the cardinality of each bid
cardinality = dict()
for bid in BidsDf.index:
    cardinality[bid] = deltaDf[bid].sum()
#cardinality

In [ ]:
#Initialize the cardinality parameter
model.cardinality = Param(model.BIDS, initialize = cardinality, doc='Number of lanes in a bid package, |S_b|')

In [ ]:
#Define the decision variable
model.x = Var(model.BIDS, domain = Binary, doc='Decision variable for each bid in the program')

In [ ]:
#Objective minimizes the sum of x_b * v_b over all bids
def obj_expression(model):
    return sum(model.bidValue[i]*model.x[i] for i in model.BIDS)
model.OBJ = Objective(rule=obj_expression, sense=minimize, doc='Objective function definition')

In [ ]:
#Define constraints
def constraint_rule(model, l):
    return sum(model.delta[l,b]*model.x[b] for b in model.BIDS) <= model.demand[l]
model.xConstraint = Constraint(model.LANES, rule=constraint_rule)

def demand_constraint_rule(model):
    return sum(model.x[b]*model.cardinality[b] for b in model.BIDS) >= model.numItems
model.demandConstraint = Constraint(rule=demand_constraint_rule)

In [ ]:
#Display of the output in order to retrieve and use in python
def pyomo_postprocess(options=None, instance=None, results=None):
    model.x.display()

In [ ]:
#Run the model
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("glpk")
%timeit results = opt.solve(model)

In [ ]:
results = opt.solve(model)
model.solutions.store_to(results)
print(results)

In [ ]:
#sends results to stdout
results.write()
print("\nDisplaying Solution\n" + '-'*60)
pyomo_postprocess(None, model, results)

In [ ]:
model.pprint()

In [ ]:
winningBids = []
index = 0
bidNum = 0
for bids in range(2199):
    if model.x[bids].value > 0:
        winningBids.append(bidNum)
    bidNum += 1
    index += 1

In [ ]:
len(winningBids) #Quick check of the length of winning bids.

In [ ]:
winningBidsDf = BidsDf.iloc[winningBids]
winningBidsDf.head()

In [ ]:
Total = winningBidsDf['Cost'].sum()
Total

In [ ]:
sum(winningBidsDf['Cost']*winningBidsDf['# Shipments/Lot'])

In [ ]:
winningBidsDf.to_csv('WinningBids.csv')